## IMPORT


In [ ]:
import requests
import xml.etree.ElementTree as ET
import csv

## Transformer fichier XML en CSV :

In [ ]:

# Chemins des fichiers
xml_file = "Test 101.xml"
csv_file = "resultat.csv"

# Parse le fichier XML
tree = ET.parse(xml_file)
root = tree.getroot()

# Supposons que chaque enfant de root est un enregistrement (ex: <person>)
records = []

for element in root:
    record = {}
    for child in element:
        record[child.tag] = child.text
    records.append(record)

# Écrit les données en CSV
if records:
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=records[0].keys())
        writer.writeheader()
        writer.writerows(records)

print(f"Fichier CSV généré : {csv_file}")

## APPEL API - TOKEN USER

In [ ]:

MAIL_USER = 'legolaslewagon@gmail.com'
# USER_NAME = 'Legolas2007'
# PASSWORD = 'Lewagon2007'
KEY_USER = '91e7670a51c5f4decf0dc3cd270c973d'  # KEYAPI NICOLAS

def user_token(USER_NAME,PASSWORD):

    URL = "https://rebrickable.com/api/v3/users/_token/"

    data = {
        'username': USER_NAME,
        'password': PASSWORD
    }

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'Authorization': f'key {KEY_USER}'
    }

    response = requests.post(URL, data=data, headers=headers)

    if response.status_code == 200:
        return response.json().get('user_token')
    else:
        print(f"Erreur {response.status_code} : {response.text}")

In [ ]:
user_token = user_token('Legolas2007','Lewagon2007')

## APPEL API - get /api/v3/lego/parts/

In [ ]:
# API_KEY = "69f550eb60a11f351c32116d2b49a075"
# URL = "https://rebrickable.com/api/v3/lego/parts/"

# headers = {
#     "Authorization": f"key {API_KEY}"
# }

# def find_part_by_bricklink_id(bricklink_id):
#     params = {
#         "bricklink_id": bricklink_id
#     }
#     response = requests.get(URL, headers=headers, params=params)
#     response.raise_for_status()
#     return response.json()

# if __name__ == "__main__":
#     bricklink_id = input("Entrez le BrickLink ID de la pièce : ").strip()
#     result = find_part_by_bricklink_id(bricklink_id)

#     if result["count"] == 0:
#         print("❌ Aucune pièce trouvée pour ce BrickLink ID.")
#     else:
#         print(f"✅ {result['count']} pièce(s) trouvée(s) pour BrickLink ID '{bricklink_id}':\n")
#         for part in result["results"]:
#             print(f"- Rebrickable ID : {part['part_num']}")
#             print(f"  Nom             : {part['name']}")
#             print(f"  Catégorie ID    : {part['part_cat_id']}")
#             print(f"  Image           : {part['part_img_url']}\n")

In [ ]:
# ici ne complète pas le CSV , il l'écrase !

# import requests
# import csv
# import time
# import os

# API_KEY = "91e7670a51c5f4decf0dc3cd270c973d"
# URL = "https://rebrickable.com/api/v3/lego/parts/"
# HEADERS = {"Authorization": f"key {API_KEY}"}

# def export_bricklink_to_rebrickable_csv(bricklink_ids, output_file="list_parts.csv"):
#     all_results = []
#     max_ids_per_row = 0

#     for idx, bl_id in enumerate(bricklink_ids):
#         print(f"[{idx+1}/{len(bricklink_ids)}] Recherche : {bl_id}")
#         try:
#             params = {"bricklink_id": bl_id}
#             response = requests.get(URL, headers=HEADERS, params=params)
#             response.raise_for_status()
#             data = response.json()
#             results = data.get("results", [])
#             rebrickable_ids = [part["part_num"] for part in results] if results else []
#         except Exception as e:
#             print(f"❌ Erreur pour {bl_id} : {e}")
#             rebrickable_ids = []

#         max_ids_per_row = max(max_ids_per_row, len(rebrickable_ids))
#         all_results.append([bl_id] + rebrickable_ids)
#         time.sleep(0.5)

#     header = ["Bricklink_ID"] + [f"Rebrickable_ID_{i+1}" for i in range(max_ids_per_row)]

#     file_exists = os.path.isfile(output_file)

#     with open(output_file, mode="a" if file_exists else "w", newline="", encoding="utf-8") as csvfile:
#         writer = csv.writer(csvfile)
#         if not file_exists:
#             writer.writerow(header)  # écrire l’en-tête uniquement si le fichier est nouveau
#         for row in all_results:
#             row += [""] * (len(header) - len(row))
#             writer.writerow(row)

#     print(f"\n✅ Fichier CSV {'complété' if file_exists else 'créé'} : {output_file}")

In [ ]:
import requests
import csv
import time
import os

API_KEY = "91e7670a51c5f4decf0dc3cd270c973d"
URL = "https://rebrickable.com/api/v3/lego/parts/"
HEADERS = {"Authorization": f"key {API_KEY}"}

def export_bricklink_to_rebrickable_csv(bricklink_ids, output_file="list_parts.csv"):
    """
    Recherche les identifiants Rebrickable correspondant à une liste d'IDs Bricklink,
    et exporte les résultats dans un fichier CSV.

    Fonctionnalités :
    -----------------
    - Si un Bricklink ID est déjà présent dans le fichier CSV, la quantité associée est incrémentée de 1.
    - Sinon, une nouvelle ligne est ajoutée avec les informations correspondantes.
    - Les colonnes sont ordonnées comme suit : Bricklink_ID, color_id, quantity, Rebrickable_ID_1, Rebrickable_ID_2, ...
    - Le color_id est fixé à 9999, et la quantité initiale à 1 pour les nouveaux éléments.

    Format du CSV :
    ---------------
    - Bricklink_ID : identifiant d'origine (string)
    - color_id : valeur fixe (int = 9999)
    - quantity : nombre d’occurrences de cette pièce (int)
    - Rebrickable_ID_N : un ou plusieurs identifiants Rebrickable (string)

    Paramètres :
    ------------
    bricklink_ids : list[str]
        Liste des IDs Bricklink à rechercher via l'API Rebrickable.
    output_file : str, optionnel (défaut="list_parts.csv")
        Nom du fichier CSV à créer ou mettre à jour.

    Retour :
    --------
    None
        La fonction écrit ou met à jour un fichier CSV sur le disque avec les résultats obtenus.
    """
    all_results = []
    existing_data = {}
    max_ids_per_row = 0

    # Étape 1 : Lire les données existantes (s'il y a un fichier)
    if os.path.isfile(output_file):
        with open(output_file, newline="", encoding="utf-8") as f:
            reader = csv.reader(f)
            header = next(reader)
            for row in reader:
                bl_id = row[0]
                color_id = row[1]
                quantity = int(row[2])
                rebrickable_ids = row[3:]
                existing_data[bl_id] = [bl_id, int(color_id), quantity] + rebrickable_ids
                max_ids_per_row = max(max_ids_per_row, len(rebrickable_ids))

    for idx, bl_id in enumerate(bricklink_ids):
        print(f"[{idx+1}/{len(bricklink_ids)}] Recherche : {bl_id}")
        try:
            params = {"bricklink_id": bl_id}
            response = requests.get(URL, headers=HEADERS, params=params)
            response.raise_for_status()
            data = response.json()
            results = data.get("results", [])
            rebrickable_ids = [part["part_num"] for part in results] if results else []
        except Exception as e:
            print(f"❌ Erreur pour {bl_id} : {e}")
            rebrickable_ids = []

        max_ids_per_row = max(max_ids_per_row, len(rebrickable_ids))

        if bl_id in existing_data:
            existing_data[bl_id][2] += 1  # Incrémenter la quantité
        else:
            existing_data[bl_id] = [bl_id, 9999, 1] + rebrickable_ids

    # Réécriture complète avec header dans l’ordre souhaité
    header = ["Bricklink_ID", "color_id", "quantity"] + [f"Rebrickable_ID_{i+1}" for i in range(max_ids_per_row)]

    with open(output_file, mode="w", newline="", encoding="utf-8") as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(header)
        for row in existing_data.values():
            # Compléter avec des colonnes vides si nécessaire
            row += [""] * (len(header) - len(row))
            writer.writerow(row)

    print(f"\n✅ Fichier CSV mis à jour : {output_file}")

In [ ]:
bricklink_ids = ["3004", "3006", "2745", "9959"]
export_bricklink_to_rebrickable_csv(bricklink_ids, output_file="list_parts.csv")

## extraire uniquement la colonne 2 Rebrickable Id 


In [ ]:
import csv

def extract_rebrickable_id1_column(csv_file="NOM DU FICHIER.csv"):
    """
    Extrait la colonne 'Rebrickable_ID_1' depuis un fichier CSV.
    Retourne une liste des IDs (non vides).
    """
    list_rebrickable_ids = []
    with open(csv_file, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            value = row.get("Rebrickable_ID_1", "").strip()
            if value:
                list_rebrickable_ids.append(value)
    return list_rebrickable_ids

In [ ]:
extract_rebrickable_id1_column(csv_file="list_parts.csv")

In [ ]:
# avec le CSV obtenu = creer une part list sur Rebrickable

In [ ]:
MAIL_USER = 'legolaslewagon@gmail.com'
USER_NAME = 'Legolas2007'
PASSWORD = 'Lewagon2007'
KEY_USER = '91e7670a51c5f4decf0dc3cd270c973d'  # KEYAPI NICOLAS
URL = "https://rebrickable.com/api/v3/users/_token/"

def get_user_token(USER_NAME,PASSWORD):

    URL = "https://rebrickable.com/api/v3/users/_token/"

    data = {
        'username': USER_NAME,
        'password': PASSWORD
    }

    headers = {
        'Content-Type': 'application/x-www-form-urlencoded',
        'Accept': 'application/json',
        'Authorization': f'key {KEY_USER}'
    }

    response = requests.post(URL, data=data, headers=headers)

    if response.status_code == 200:
        user_token = response.json().get('user_token')
        return user_token
    else:
        print(f"Erreur {response.status_code} : {response.text}")


get_user_token(USER_NAME,PASSWORD)

## Créer une list part sur ton compte

In [ ]:
from requests.exceptions import HTTPError

user_token = get_user_token(USER_NAME,PASSWORD)
KEY_USER = '91e7670a51c5f4decf0dc3cd270c973d'

def create_partlist(user_token, name_new_list,list_type=1):
    """
    Crée une nouvelle Part List.
    - name (str): nom de la liste.
    - list_type (int): 1 = utilisée pour Build, 2 = non utilisée dans Build.

    Retourne : JSON avec le détail de la liste créée.
    """
    url = f"https://rebrickable.com/api/v3/users/{user_token}/partlists/"
    headers = {"Authorization": f"key {KEY_USER}"}
    payload = {"name": name_new_list , "type": list_type}

    try:
        resp = requests.post(url, headers=headers, json=payload)
        resp.raise_for_status()
        data_list= resp.json()
        return data_list

    except HTTPError as http_err:
        if resp.status_code == 400 or resp.status_code == 409:
            # Souvent erreur 409 pour conflit / doublon
            print(f"Erreur : une liste avec le nom '{name_of_new_list}' existe déjà.")
        else:
            print(f"Erreur HTTP {resp.status_code} : {resp.text}")
        return None

In [ ]:

create_partlist(user_token,"test1",list_type=1)

In [ ]:
def get_id_list(name_new_list):
    data_list = create_partlist(user_token, name_new_list ,list_type=1)
    id_list = data_list.get('id')
    return id_list

In [ ]:
name_new_list ="test3"
id_list = get_id_list(name_new_list)

In [ ]:
id_list


## API pour supprimer une liste ID OBLIGATOIRE 

In [ ]:
KEY_USER = '91e7670a51c5f4decf0dc3cd270c973d'

def delete_partlist(user_token, id_list):
    url = f"https://rebrickable.com/api/v3/users/{user_token}/partlists/{id_list}/"
    headers = {"Authorization": f"key {KEY_USER}"}

    response = requests.delete(url, headers=headers)

    if response.status_code == 204:
        print(f"✅ Partlist {id_list} supprimée avec succès.")
    else:
        print(f"❌ Erreur {response.status_code} lors de la suppression : {response.text}")

In [ ]:
delete_partlist(user_token,id_list)

## API pour compléter une liste

In [ ]:
# extraire la liste des rebrikable ID

list_rebrickable_ids = extract_rebrickable_id1_column(csv_file="list_parts.csv")
list_rebrickable_ids


In [ ]:
import csv
import json

def csv_to_json_parts(csv_file):
    """
    Lit un fichier CSV et retourne une liste de dicts JSON au format attendu par l'API Rebrickable.
    Chaque dict contient les clés : 'part_num', 'color_id', 'quantity'.
    Utilise la colonne 'Rebrickable_ID_1' pour 'part_num'.

    Paramètres :
    -----------
    csv_file : str
        Chemin vers le fichier CSV à lire.

    Retourne :
    ---------
    list[dict]
        Liste de dictionnaires JSON correspondant aux pièces.
    """
    parts_list = []
    with open(csv_file, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for row in reader:
            part_num = row.get("Rebrickable_ID_1")
            color_id = row.get("color_id")
            quantity = row.get("quantity")

            # Convertir color_id et quantity en int si possible, sinon valeur par défaut
            try:
                color_id = int(color_id)
            except (ValueError, TypeError):
                color_id = 9999
            try:
                quantity = int(quantity)
            except (ValueError, TypeError):
                quantity = 1

            if part_num:  # On vérifie qu'il y a bien un ID Rebrickable
                parts_list.append({
                    "part_num": part_num,
                    "color_id": color_id,
                    "quantity": quantity
                })
    return parts_list


In [ ]:
json_parts = csv_to_json_parts("list_parts.csv")
json_parts

## Ajouter parts to list


In [ ]:
import requests

def add_parts_to_partlist(user_token, id_list, json_parts, KEY_USER):
    """
    Ajoute plusieurs parts à une partlist Rebrickable.

    Args:
        user_token (str): Le token utilisateur Rebrickable.
        list_id (int): L'ID de la partlist où ajouter les parts.
        parts (list of dict): Liste de parts au format [{"part_num": str, "color_id": int, "quantity": int}, ...]
        api_key (str): Clé API Rebrickable.

    Returns:
        list: Liste des parts ajoutées avec succès (réponse JSON).
    """
    url = f"https://rebrickable.com/api/v3/users/{user_token}/partlists/{id_list}/parts/"
    headers = {
        "Authorization": f"key {KEY_USER}",
        "Content-Type": "application/json"
    }

    response = requests.post(url, headers=headers, json=json_parts)
    response.raise_for_status()
    return response.json()

In [ ]:
add_parts_to_partlist(user_token, id_list, json_parts, KEY_USER)

In [ ]:
add_parts_to_partlist(user_token, id_list, parts, KEY_USER)

## test set

In [ ]:
import requests
import pandas as pd

def can_build_set(set_num):
    """
    Vérifie si l'utilisateur peut construire un set donné avec ses pièces.

    Args:
        user_token (str): Token utilisateur Rebrickable.
        set_num (str): Numéro du set (ex: '75301-1').
        api_key (str): Clé API Rebrickable.

    Returns:
        dict: Détails de la construction possible (pièces manquantes, % de complétion, etc.)
    """
    url = f"https://rebrickable.com/api/v3/users/{user_token}/build/{set_num}/"
    headers = {
        "Authorization": f"key {KEY_USER}"
    }

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Lève une erreur si la requête échoue
    result = response.json()

    return result


In [ ]:
set_num = "75301-1" # string

In [ ]:
def do_set_with_myparts(set_num):
    result=can_build_set(set_num)
    info = {" Set":[set_num],
            "Pieces total du set": [result.get('total_parts')],
        "% pièces possédée du set" : [round(result.get('pct_owned'),1)],
        "Nombre pièce manquante" : [result.get('num_missing')],
        }

    df=pd.DataFrame(info)
    return df

In [ ]:
do_set_with_myparts(set_num)

In [ ]:
info = {" Set":['set_num'],
"% pièces possédée du set" : [result.get('pct_owned')],
"Nombre pièce manquante" : [result.get('num_missing')],
}

df=pd.DataFrame(info)


In [ ]:
df

In [ ]:
result.get('num_missing')

In [ ]:
import requests
import pandas as pd

set_num = "75301-1"  # string
KEY_USER = '91e7670a51c5f4decf0dc3cd270c973d'

def part_set(set_num):
    url = f"https://rebrickable.com/api/v3/lego/sets/{set_num}/parts/"
    headers = {
        "Authorization": f"key {KEY_USER}"
    }

    rows = []
    while url:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Lève une erreur si la requête échoue
        result = response.json()

        for item in result["results"]:
            # Ne pas prendre les pièces de rechange
            if item.get("is_spare", False):
                continue

            part_num = item["part"]["part_num"]
            quantity = item["quantity"]
            color = item["color"]["id"]
            rows.append({"part_num": part_num, "quantity": quantity, "color": color})

        url = result.get("next")  # Passe à la page suivante si elle existe, sinon None

    df = pd.DataFrame(rows)
    return df

In [ ]:
df = part_set(set_num)

nombre_parts = len(df)
total_pieces = df["quantity"].sum()

print(f"Le set {set_num} contient {nombre_parts} types de pièces différents.")
print(f"Le set {set_num} contient au total {total_pieces} pièces (quantités comprises) hors rechange.")


In [ ]:
df

In [ ]:
result = part_set(set_num)
result.get('results')

In [ ]:
parts = []
parts_num = []

for item in result["results"]:
    part = item["part"]
    parts.append(part)
    parts_num.append(part["part_num"])


CREATION fonction API couleur

In [ ]:
def part_color(part_num):

    url='https://rebrickable.com/api/v3/lego/parts/{part_num}/colors/'

    HEADERS = {'Accept': 'application/json',
               "Authorization": f"key {KEY_USER}"
               }


    response = requests.get(url,headers=HEADERS)

    if response.status_code == 200:
        data=response.json()
        for item in data['results']:
            colors = item['color']['name']
        return {part_num:colors}
    else:
        print(f"Erreur {response.status_code} : {response.text}")


In [12]:
import requests
KEY_USER = "91e7670a51c5f4decf0dc3cd270c973d"
colors = part_color('3001')

In [1]:
from main_api import part_color,get_rebrickable_id

part_color('3001')

,bricklink_id,rebrickable_id,color_id,color_name,num_sets,num_set_parts,part_img_url,elements
0,3001,3001,0,Black,1048,4537,https://cdn.rebrickable.com/media/parts/elemen...,300126
1,3001,3001,1,Blue,645,4465,https://cdn.rebrickable.com/media/parts/elemen...,"300123, 300173"
2,3001,3001,2,Green,355,2051,https://cdn.rebrickable.com/media/parts/elemen...,"300128, 4106356"
3,3001,3001,3,Dark Turquoise,51,583,https://cdn.rebrickable.com/media/parts/elemen...,"4143667, 6249422"
4,3001,3001,4,Red,956,6215,https://cdn.rebrickable.com/media/parts/elemen...,300121
...,...,...,...,...,...,...,...,...
61,3001,3001,1050,Coral,12,61,https://cdn.rebrickable.com/media/parts/elemen...,6422921
62,3001,3001,1051,Pastel Blue,4,17,https://cdn.rebrickable.com/media/parts/elemen...,"300111, 4112545, 4162339, 4250614, 4585679"
63,3001,3001,1062,Vibrant Yellow,4,23,https://cdn.rebrickable.com/media/parts/elemen...,6427901
64,3001,3001,1091,Warm Yellowish Orange,1,1,https://cdn.rebrickable.com/media/parts/elemen...,4243872
